<a href="https://colab.research.google.com/github/kutluhanNG/MLcodes/blob/main/BTC_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests gradio pandas-ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 115.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 7.6 MB/s eta 0:00:00
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=2e4050ef6c0697c8e477361e6fcf4bc4b89df06b2341fa5a5336fa77940e436e
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas-ta
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling Mark

In [2]:
import requests
import pandas as pd
import time
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
import gradio as gr
import pandas_ta as ta
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
# Function to fetch large cryptocurrency data from CoinGecko API
def get_large_crypto_data(symbol="bitcoin", days=360):
    url = f"https://api.coingecko.com/api/v3/coins/{symbol}/market_chart"
    params = {"vs_currency": "usd", "days": days}
    response = requests.get(url, params=params)
    data = response.json()
    prices = data["prices"]

    # Convert to DataFrame
    df = pd.DataFrame(prices, columns=["timestamp", "close"])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    return df

In [4]:
# Fetch data for Bitcoin over the past 360 days
df = get_large_crypto_data("bitcoin", days=360)

In [5]:
# Check the data type and missing values in the "close" column
print(df["close"].dtypes)  # Print the data type of the "close" column
print(df["close"].isna().sum())  # Print the number of missing values in the "close" column

# Fill missing values before adding indicators
df["close"].fillna(method="ffill", inplace=True)  # Forward fill missing values in the "close" column
df["close"] = df["close"].astype(float)  # Convert the "close" column to float type

# Add technical indicators back to the DataFrame
df["SMA_10"] = ta.sma(df["close"], length=10)  # Calculate the 10-day Simple Moving Average (SMA)
df["EMA_10"] = ta.ema(df["close"], length=10)  # Calculate the 10-day Exponential Moving Average (EMA)
df["RSI"] = ta.rsi(df["close"], length=14)  # Calculate the 14-day Relative Strength Index (RSI)
df["MACD"], df["MACD_signal"] = ta.macd(df["close"]).iloc[:, 0:2]  # Calculate MACD and its signal line
df.fillna(0, inplace=True)  # Replace any remaining missing values with 0

float64
0


<ipython-input-5-b9d9330f47c6>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["close"].fillna(method="ffill", inplace=True)  # Forward fill missing values in the "close" column
<ipython-input-5-b9d9330f47c6>:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["close"].fillna(method="ffill", inplace=True)  # Forward fill missing values in the "close" column


In [6]:
# Check the data types of the columns for MinMaxScaler
print(df[["close", "SMA_10", "EMA_10", "RSI", "MACD"]].dtypes)

# If there are strings or missing values in these columns, fix them
df[["close", "SMA_10", "EMA_10", "RSI", "MACD"]] = df[["close", "SMA_10", "EMA_10", "RSI", "MACD"]].apply(pd.to_numeric, errors="coerce")
df.fillna(0, inplace=True)  # Replace missing values with 0

# Apply MinMaxScaler to normalize the data between 0 and 1
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_prices = scaler.fit_transform(df[["close", "SMA_10", "EMA_10", "RSI", "MACD"]])

close     float64
SMA_10    float64
EMA_10    float64
RSI       float64
MACD       object
dtype: object


In [7]:
# Function to prepare data for LSTM model
def create_lstm_dataset(data, time_step, future_days):
    X, Y = [], []
    for i in range(len(data) - time_step - future_days):
        X.append(data[i:(i + time_step), :])  # Input features
        Y.append(data[(i + time_step):(i + time_step + future_days), 0])  # Target variable (close prices)
    return np.array(X), np.array(Y)

# Define parameters
time_step = 50
future_days = 10

# Create LSTM dataset
X, Y = create_lstm_dataset(scaled_prices, time_step, future_days)
train_size = int(len(X) * 0.8)  # 80% of data for training
X_train, X_test = X[:train_size], X[train_size:]
Y_train, Y_test = Y[:train_size], Y[train_size:]

In [8]:
# Use TensorFlow MirroredStrategy for multi-GPU support
strategy = tf.distribute.MirroredStrategy()
print(f"Number of GPUs in use: {strategy.num_replicas_in_sync}")

Number of GPUs in use: 1


In [9]:
# Build and compile the LSTM model within the strategy scope
with strategy.scope():
    model = Sequential([
        Input(shape=(time_step, X.shape[2])),
        LSTM(units=128, return_sequences=True),
        Dropout(0.2),
        LSTM(units=128, return_sequences=False),
        Dropout(0.2),
        Dense(units=50),
        Dense(units=future_days)  # Predicting multiple future days
    ])

    model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])

In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 50, 128)             │          68,608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 50, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 50)                  │           6,450 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │             510 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 207,152 (809.19 KB)

 Trainable params: 207,152 (809.19 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# ModelCheckpoint callbacks
model_checkpoint = ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss")

In [12]:
# Train the model
history = model.fit(
    X_train, Y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test, Y_test),
    callbacks=[model_checkpoint],
    verbose=1
)

Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 111ms/step - loss: 0.0599 - mae: 0.1913 - val_loss: 0.2516 - val_mae: 0.4914
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.0249 - mae: 0.1060 - val_loss: 0.0995 - val_mae: 0.3039
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0209 - mae: 0.1084 - val_loss: 0.1017 - val_mae: 0.2989
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.0155 - mae: 0.0889 - val_loss: 0.0172 - val_mae: 0.1072
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0133 - mae: 0.0872 - val_loss: 0.0486 - val_mae: 0.1883
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.0123 - mae: 0.0875 - val_loss: 0.0135 - val_mae: 0.0928
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0106 - mae: 0.0798 - val_loss: 0.0411 - val_mae: 0.1816
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0102 - mae: 0.0731 - val_loss: 0.0272 - val_mae: 0.1389
Epoch 9/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.0096 - mae: 0.0731 -

In [18]:
# Save the trained model
model.save("coin_gpt_lstm_large.keras")

In [20]:
model = tf.keras.models.load_model("best_model.keras")
X_input = scaled_prices[-time_step:].reshape(1, time_step, X.shape[2])
predicted_prices = model.predict(X_input)
print("Predicted Prices Before Reshaping:", predicted_prices)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
Predicted Prices Before Reshaping: [[0.8235842  0.8261941  0.81214607 0.8382349  0.82552403 0.83089614
  0.8492996  0.8197435  0.8345424  0.851429  ]]


In [21]:
# (10, 5) boyutlarında bi matrix oluştur hoca
dummy = np.zeros((predicted_prices.shape[1], X.shape[2]))  # shape: (10, 5)

# ilk sütünu tahmin edilen veriyle doldur (yukarıdan aşağı 10 tane olacak yani)
dummy[:, 0] = predicted_prices.flatten()

predicted_close = scaler.inverse_transform(dummy)[:, 0]

print("10-Day Bitcoin Price Predictions:")
for i, price in enumerate(predicted_close, start=1):
    print(f"Day {i}: {round(price, 2)} USD")


10-Day Bitcoin Price Predictions:
Day 1: 96223.55 USD
Day 2: 96370.88 USD
Day 3: 95577.87 USD
Day 4: 97050.58 USD
Day 5: 96333.05 USD
Day 6: 96636.31 USD
Day 7: 97675.19 USD
Day 8: 96006.74 USD
Day 9: 96842.14 USD
Day 10: 97795.39 USD


In [22]:
dummy

array([[0.8235842 , 0.        , 0.        , 0.        , 0.        ],
       [0.82619411, 0.        , 0.        , 0.        , 0.        ],
       [0.81214607, 0.        , 0.        , 0.        , 0.        ],
       [0.8382349 , 0.        , 0.        , 0.        , 0.        ],
       [0.82552403, 0.        , 0.        , 0.        , 0.        ],
       [0.83089614, 0.        , 0.        , 0.        , 0.        ],
       [0.84929961, 0.        , 0.        , 0.        , 0.        ],
       [0.81974351, 0.        , 0.        , 0.        , 0.        ],
       [0.83454239, 0.        , 0.        , 0.        , 0.        ],
       [0.85142899, 0.        , 0.        , 0.        , 0.        ]])